# Crawl von ACDH-CH-HowTo

- Website: https://howto.acdh.oeaw.ac.at/de/resources/posts/page/1 (Auflistung aller Tutorials)
- Ziel: Extraktion von Tags + Anzahl + Titel von Beiträgen, die mit einem Topic getaggt sind

In [5]:
import requests
from bs4 import BeautifulSoup
import csv
import json
from urllib.parse import urljoin
from collections import Counter

In [2]:
url = "https://howto.acdh.oeaw.ac.at/de/resources/posts/page/1"
r = requests.get(url)
doc = BeautifulSoup(r.text, "html.parser")

In [12]:
#Titel und Links pro Post von Überblicksseite crawlen

ergebnis = {}
counter = 0

for article in doc.find_all("article"):
    for h3 in article.select("h3 a"):
        titel = h3.text
        link = h3.attrs["href"]
        ergebnis[counter] = {"titel": titel, "link": link}
        counter = counter + 1

print(ergebnis)

{0: {'titel': 'Langzeitarchivierung', 'link': '/de/resource/posts/langzeitarchivierung'}, 1: {'titel': 'Einführung in Metadaten und Dokumentation', 'link': '/de/resource/posts/einfuehrung-metadaten'}, 2: {'titel': 'Praktische Übung: Grundlagen des Datenmanagements', 'link': '/de/resource/posts/datenmanagement-%C3%BCbung'}, 3: {'titel': 'Grundlagen des Datenmanagements', 'link': '/de/resource/posts/grundlagen-datenmanagement'}, 4: {'titel': 'Introduction to Dictionary Users', 'link': '/de/resource/posts/introduction-to-dictionary-users'}, 5: {'titel': 'A Short Introduction to Figma', 'link': '/de/resource/posts/introduction-to-figma'}, 6: {'titel': 'Creating Lexical Resources with the <TEI>Enricher', 'link': '/de/resource/posts/a-very-concise-guide-to-creating-lexical-resources-with-tei-enricher'}, 7: {'titel': 'Prosopographic Modelling - Introduction', 'link': '/de/resource/posts/prosopographic-modelling-introduction'}, 8: {'titel': 'Prosopographic Modelling - Individuals', 'link': '/d

In [15]:
#von relativen zu funktionalen Links

for key in ergebnis.keys():
    link_neu = urljoin("https://howto.acdh.oeaw.ac.at/de/resources/posts/page/1", ergebnis[key]["link"])
    ergebnis[key]["link"] = link_neu
    print(link_neu)

https://howto.acdh.oeaw.ac.at/de/resource/posts/langzeitarchivierung
https://howto.acdh.oeaw.ac.at/de/resource/posts/einfuehrung-metadaten
https://howto.acdh.oeaw.ac.at/de/resource/posts/datenmanagement-%C3%BCbung
https://howto.acdh.oeaw.ac.at/de/resource/posts/grundlagen-datenmanagement
https://howto.acdh.oeaw.ac.at/de/resource/posts/introduction-to-dictionary-users
https://howto.acdh.oeaw.ac.at/de/resource/posts/introduction-to-figma
https://howto.acdh.oeaw.ac.at/de/resource/posts/a-very-concise-guide-to-creating-lexical-resources-with-tei-enricher
https://howto.acdh.oeaw.ac.at/de/resource/posts/prosopographic-modelling-introduction
https://howto.acdh.oeaw.ac.at/de/resource/posts/prosopographic-modelling-individuals
https://howto.acdh.oeaw.ac.at/de/resource/posts/prosopographic-modelling-ontologies
https://howto.acdh.oeaw.ac.at/de/resource/posts/prosopographic-modelling-vocabularies


In [20]:
#Tags aus einzelnen Tutorialseiten crawlen

for key in ergebnis.keys():
    url = ergebnis[key]["link"]
    r = requests.get(url)
    doc = BeautifulSoup(r.text, "html.parser")
    for dd in doc.select_one("dd"):
        tags = dd.text.strip().split(",")
    ergebnis[key]["tags"] = tags.strip()

for key in ergebnis.keys():
    print(ergebnis[key]) 
    

{'titel': 'Langzeitarchivierung', 'link': 'https://howto.acdh.oeaw.ac.at/de/resource/posts/langzeitarchivierung', 'tags': ['Datenmanagement']}
{'titel': 'Einführung in Metadaten und Dokumentation', 'link': 'https://howto.acdh.oeaw.ac.at/de/resource/posts/einfuehrung-metadaten', 'tags': ['Datenmanagement']}
{'titel': 'Praktische Übung: Grundlagen des Datenmanagements', 'link': 'https://howto.acdh.oeaw.ac.at/de/resource/posts/datenmanagement-%C3%BCbung', 'tags': ['Datenmanagement']}
{'titel': 'Grundlagen des Datenmanagements', 'link': 'https://howto.acdh.oeaw.ac.at/de/resource/posts/grundlagen-datenmanagement', 'tags': ['Datenmanagement']}
{'titel': 'Introduction to Dictionary Users', 'link': 'https://howto.acdh.oeaw.ac.at/de/resource/posts/introduction-to-dictionary-users', 'tags': ['Lexicography', ' Linguistics']}
{'titel': 'A Short Introduction to Figma', 'link': 'https://howto.acdh.oeaw.ac.at/de/resource/posts/introduction-to-figma', 'tags': ['UI design']}
{'titel': 'Creating Lexical

In [27]:
#Tags zusammensammeln und -zählen

tags = []

for key in ergebnis.keys():
    for tag in ergebnis[key]["tags"]:
        tags.append(tag)

tagCounter = Counter(tags)

print(tagCounter)

Counter({'Datenmanagement': 4, 'Prosopographic data': 4, ' Semantic web': 4, 'Lexicography': 1, ' Linguistics': 1, 'UI design': 1, 'TEI': 1})


In [28]:
#externe Speicherung der beiden Dictionaries (Post- vs. Taglevel)

with open("HowTo_Posts.json", "w", encoding = "utf-8") as datei:
    json.dump(ergebnis, datei, ensure_ascii = False)

with open("HowTo_Tags.json", "w", encoding = "utf-8") as datei:
    json.dump(tagCounter, datei, ensure_ascii = False)

In [30]:
#Speicherung als .csv-Dateien (erneut: Post- vs. Taglevel)

with open("HowTo_Posts.csv", "w", newline = "", encoding = "utf-8") as datei:
    writer = csv.writer(datei, delimiter=';', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(["Titel", "Link", "Tags"])
    for key in ergebnis.keys():
        tags_formatiert = ", ".join(ergebnis[key]["tags"])
        writer.writerow(
            [ergebnis[key]["titel"], 
             ergebnis[key]["link"],
             tags_formatiert])

with open("HowTo_Tags.csv", "w", newline = "", encoding = "utf-8") as datei:
    writer = csv.writer(datei, delimiter=';', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(["Tag", "Anzahl"])
    for key in tagCounter.keys():
        writer.writerow(
            [key, 
             tagCounter[key]])

### Weitere Tags im CMS (oben nicht mitaufgenommen)

- CMS für HowTo: von ACDH-CH-Github-Mitgliedern einsehbar/bearbeitbar - https://github.com/acdh-oeaw/howto 
- dort **10 zusätzliche Tags** auffindbar, die im Frontend noch nicht aufscheinen
- 3 Tags bereits unter "Content", aber (noch) für keinen Beitrag in Verwendung:
    - "Corpus"
    - "Documentation HowTo Platform"
    - "XML"
- 7 Tags als "Ready" im "Editorial Workflow", d.h. diese könnten bald publiziert werden (laut Susanne Zhanial jedoch noch unklar, ob dies passiert, da langfristig ohnehin Orientierung an DARIAH-Campus angestrebt):
    - "Ontology"
    - "Linked Open Data"
    - "Controlled vocabulary"
    - "SKOS"
    - "Open acess"
    - "SPARQL"
    - "RDF"

In [3]:
#Excel-Tabelle (s. gesendetes Dokument) um Tags und Descriptions aus CMS ergänzt - hier diese Daten zur Weiterverarbeitung importiert
#erneutes Durchführen der Zelle NICHT notwendig

ht = {}
counter = 0

with open("HowTo_ergänzt.csv", encoding = "utf-8") as datei:
    for row in datei:
        if counter == 0:
            counter = counter + 1
            continue
        else:
            zs = row.split(";")
            tag = zs[0].strip()
            description = zs[1].strip()
            count = zs[2].strip()
            ht[tag] = {"description": description, "count": count}

print(ht)

{'Datenmanagement': {'description': 'Datenmanagement', 'count': '4'}, 'Prosopographic data': {'description': 'Prosopographic data', 'count': '4'}, 'Semantic web': {'description': 'Semantic web', 'count': '4'}, 'Lexicography': {'description': 'The theory and practice of compiling dictionaries.', 'count': '1'}, 'Linguistics': {'description': 'Linguistics', 'count': '1'}, 'UI design': {'description': 'UI (user interface) design', 'count': '1'}, 'TEI': {'description': 'TEI', 'count': '1'}, 'Ontology': {'description': 'An ontology represents entities, data and ideas through the definition of categories, their properties, their interdependencies and relations, that show how the concepts, data and entities are related to a particular subject area, i.e. discourse. An ontology limits the complexity of a subject area and groups data into information and knowledge. When a controlled vocabulary is used to talk about the information and knowledge that are represented in an ontology, communication i

In [6]:
#Speicherung als Dictionary
#erneutes Durchführen der Zelle NICHT notwendig

with open("HowTo_ergänzt.json", "w", encoding = "utf-8") as datei:
    json.dump(ht, datei, ensure_ascii = False)

### Zusammenfassung zu Tags im ACDH-CH HowTo (Stand 20.07.2022)

- **Anzahl:** 17 verschiedene Konzepte (7 davon bereits verwendet, 10 angelegt/als Entwurf)
- **Sprache:** primär Englisch, nur ein Tag eindeutig Deutsch, einzelne wie "SPARQL" oder "TEI" keiner Sprache zuordenbar, keine Übersetzungen vorhanden
- **Clustering/Hierachie:** nein
- **URIs:** nein
- **Alternativnamen:** nein
- **Descriptions**: teilweise bereits vorhanden(s. oben, im Excel oder im CMS unter Content > Tags)
- **Offenheit/Geschlossenheit**: offene Liste (in Absprache mit dem Training-Team ist es grundsätzlich erlaubt, neue Tags hinzuzufügen)
- **Schwerpunkt**: Inhalt relativ heterogen, kein klarer Schwerpunkt erkennbar --> Konzepte stammen aus den verschiedensten Bereichen:
    - z.B. Aktivitäten: "Datenmanagement"
    - z.B. Felder/Disziplinen: "Linguistics", "Lexicography"
    - z.B. (Markup-/Programmier-)Sprachen/Formate/Standards: "XML", "TEI", "SKOS"
    - z.B. Arten von wissenschaftlichem Output: "Controlled Vocabulary", "Corpus"
    - teilweise auch Einordung schwierig, da sehr spezifisch - z.B. "Documentation HowTo Platform"
